In [ ]:
from src import api
from src import wiki

url = "https://lolpros.gg/player/goksi"
player_data = api.get_player_data_by_lolpros(url, last_n=20)

In [ ]:
from constants import *

get_champ_name_to_id()

In [1]:
import psycopg2
from psycopg2 import extras
import pandas as pd
from datetime import datetime
from src import api
from src import wiki
from src.wiki import ROLE_ICONS

try:
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host="postgresql.r5.websupport.sk",
        database="brch_db",
        user="brch_admin",
        password="Ps8zV>95Dg",
        connect_timeout=600,
    )

    with conn.cursor() as cur:
        # Fetch teams data from the database
        cur.execute("SELECT id, name, queryname, url, stats FROM team")
        teams = cur.fetchall()
        teams = pd.DataFrame(teams, columns=["id", "name", "queryname", "url", "stats"])

        for _, team in teams.iterrows():
            # Handle team params
            # ----------------------------
            query = team["queryname"]
            team_id = team["id"]
            team_picks, bans, bans_against = wiki.get_team_data_from_lol_wiki(query)
            dt_string = datetime.now().strftime("%m-%d-%Y %H:%M:%S")
            stats = {"picks": team_picks, "bans": bans, "bans_against": bans_against}
            stats = extras.Json(stats)

            # Update team stats
            # ----------------------------
            update_query = """
                UPDATE team
                SET stats = %s, last_update = %s
                WHERE id = %s
            """
            cur.execute(update_query, (stats, dt_string, team_id))

            cur.execute(
                f"SELECT id, name, lolprosurl, alternativeids, role FROM player WHERE teamid = {team_id}"
            )
            team_players = cur.fetchall()
            team_players = pd.DataFrame(
                team_players,
                columns=["id", "name", "lolprosurl", "alternativeids", "role"],
            )
            for _, player in team_players.iterrows():
                url = player["lolprosurl"]  # "https://lolpros.gg/player/goksi"
                player_id = player["id"]
                alts = (
                    [] if player["alternativeids"] is None else player["alternativeids"]
                )
                player_data = api.get_player_data_by_lolpros(url, last_n=20, alts=alts)
                player_data = extras.Json(player_data)
                update_player_query = """
                UPDATE player
                SET accounts = %s, last_update = %s, icon = %s
                WHERE id = %s
                """
                cur.execute(
                    update_player_query,
                    (player_data, dt_string, ROLE_ICONS[player["role"]], player_id),
                )

            # Commit the changes to the database
            conn.commit()

except Exception as e:
    print(e)
finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()

2023-06-15 12:29:55,521 - INFO - Looking for team - stopwatch
2023-06-15 12:29:55,522 - INFO - Query url - https://lol.fandom.com/wiki/Hitpoint_2nd_Division_Challengers/2023_Season/Summer_Season/Match_History
2023-06-15 12:29:56,137 - INFO - stopwatch parsing successful
2023-06-15 12:29:57,159 - INFO - player - fogzy | ids found - ['ff in head', 'ANDRIY BAMPER']
2023-06-15 12:30:06,454 - INFO - ff in head - found 49 games
2023-06-15 12:30:06,466 - INFO - ff in head | soloq : MASTER I 618 | flexq : Unranked
2023-06-15 12:30:07,694 - INFO - ANDRIY BAMPER - found 3 games
2023-06-15 12:30:07,704 - INFO - ANDRIY BAMPER | soloq : MASTER I 260 | flexq : Unranked
2023-06-15 12:30:09,679 - INFO - player - zaki | ids found - ['běhej rychlejc', 'Žaki']
2023-06-15 12:30:11,739 - INFO - běhej rychlejc - found 8 games
2023-06-15 12:30:11,747 - INFO - běhej rychlejc | soloq : MASTER I 82 | flexq : Unranked
2023-06-15 12:30:12,545 - INFO - Žaki - found 0 games
2023-06-15 12:30:12,554 - INFO - Žaki | s